In [1]:
# | default_exp core

In [2]:
# | hide
%load_ext autoreload
%autoreload 2

import nbdev
import jupyter_black
import nbdev.showdoc as showdoc
from fastcore.test import *
from fastcore.imports import *

In [3]:
# | hide
jupyter_black.load()

In [4]:
# | export
import logging
from copy import deepcopy
from textwrap import indent
from collections import ChainMap
from typing import Any, Generator, Sequence, Optional

In [5]:
# | hide
import logging

logging.getLogger().setLevel(logging.INFO)

# Core

> Core pieces needed to use during code generation.

## Data

Out code generator will need a class to hold data while doing code generation. For this reason, the first class we are developing is creatively named `Data`

In [6]:
# | export
class Data:
    """
    Data holder used during code generation. Logic is kept as separate functions.
    """

    parent: Optional["Data"]
    children: Sequence["Data"]

    def __init__(
        self,
        name: str,  # Name of this element
        attrs: Optional[dict[str, Any]] = None,  # Attributes for this element
    ) -> None:
        """
        Initialize Data object.

        """

        self.name = name
        # TODO accept parent directly in init?
        self.parent: Optional["Data"] = None
        self.children: Sequence["Data"] = []

        # we don't directly store them because the final attributes are
        # composed by the attributes of itself and its parents.
        self._attrs = {} if attrs is None else attrs

    @property
    def attrs(self) -> ChainMap:
        """
        Get the attributes for this element, merged with
        parent's attributes, if available.
        """
        if self.parent:
            return ChainMap(self._attrs, self.parent.attrs)
        return ChainMap(self._attrs)

    def clone(self) -> "Data":
        """
        Create a deep copy of this Data object.

        """
        return deepcopy(self)

    def append(self, child: "Data") -> "Data":
        """
        Add a child element to the children list and set its parent to self.
        """
        self.children.append(child)  # type: ignore[attr-defined]
        child.set_parent(self)
        return child

    def set_parent(self, parent: "Data") -> None:
        """
        Set the parent element of self.
        """
        logging.info("Setting parent %s for %s", parent, self)
        self.parent = parent

    def __eq__(self, a: Any) -> bool:
        """
        Compare this Data object with another object for equality.

        """
        same_name: bool = self.name == a.name
        same_attrs: bool = self.attrs == a.attrs
        same_children: bool = self.children == a.children
        return same_name and same_attrs and same_children

    def as_tree(self) -> str:
        """
        Get the string representation of this Data object.

        """
        is_self_closing = not self.children

        if self.children:
            children = "\n".join(map(str, self.children))
            children = children.strip()
            children = f"\n{children}\n"
            children = indent(children, "    ")

        if self.attrs:
            if is_self_closing:
                return f"<{self.name} {dict(self.attrs)} />"
            else:
                return f"<{self.name} {dict(self.attrs)}>{children}</{self.name}>"

        if is_self_closing:
            return f"<{self.name} />"
        else:
            return f"<{self.name}>{children}</{self.name}>"

    def __str__(self) -> str:
        return f"[{self.name} {dict(self.attrs)}]"

    def __len__(self) -> int:
        return len(self.children)

    def __contains__(self, child: "Data") -> bool:
        return child in self.children

    def __iter__(self) -> Generator:
        def iter_data(obj: "Data", level: Optional[int] = 0) -> Generator:
            """Simply yields parent and then children"""
            yield obj, level
            for child in obj.children:
                yield from iter_data(child, level=(level or 0) + 1)

        return iter_data(self)

    __repr__ = as_tree

In [7]:
# | hide
showdoc.show_doc(Data)

---

[source](https://github.com/mintyPT/sal/blob/main/sal/core.py#L14){target="_blank" style="float:right; font-size:smaller"}

### Data

>      Data (name:str, attrs:Optional[dict[str,Any]]=None)

Data holder used during code generation. Logic is kept as separate functions.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| name | str |  | Name of this element |
| attrs | Optional | None | Attributes for this element |
| **Returns** | **None** |  |  |

### Basic operations

In [8]:
james = Data("person", {"name": "james"})

test_eq(james.name, "person")
test_eq(james.attrs, {"name": "james"})

We can add children (note: a child's attributes will also include those of his parent)

In [9]:
james = Data("person", {"name": "james", "root": "true"})
olive = Data("person", {"name": "olive"})
silva = Data("person", {"name": "silva"})
andrew = Data("person", {"name": "andrew"})
john = Data("person", {"name": "john"})
jane = Data("person", {"name": "jane"})
noname = Data("person", {"name": ""})

james.append(olive)
james.append(silva)
james.append(john)

olive.append(andrew)
olive.append(jane)
olive.append(noname)


# ---
test_eq(james.children[0].attrs["name"], "olive")
test_eq(jane.attrs["root"], "true")

INFO:root:Setting parent [person {'name': 'james', 'root': 'true'}] for [person {'name': 'olive'}]
INFO:root:Setting parent [person {'name': 'james', 'root': 'true'}] for [person {'name': 'silva'}]
INFO:root:Setting parent [person {'name': 'james', 'root': 'true'}] for [person {'name': 'john'}]
INFO:root:Setting parent [person {'name': 'olive', 'root': 'true'}] for [person {'name': 'andrew'}]
INFO:root:Setting parent [person {'name': 'olive', 'root': 'true'}] for [person {'name': 'jane'}]
INFO:root:Setting parent [person {'name': 'olive', 'root': 'true'}] for [person {'name': ''}]


and a child will know its parent

In [10]:
print(olive.parent.attrs["name"])

# ---
assert james == olive.parent
# test_eq(james, olive.parent)

james


To check the number of children, simply use `len`

In [11]:
len(james)

# ---
assert len(james) == 3

You can compare elements but they are tested based on their attributes and children

In [12]:
b = Data("b", {"age": 22})
c = Data("b", {"age": 22})
d = Data("d")
b.append(d)
c.append(d)

# ---
assert b == c
# test_eq(b, c)
assert Data("b", {"name": "santos"}) == Data("b", {"name": "santos"})
# test_eq(Data("b", {"name": "santos"}), Data("b", {"name": "santos"}))
assert Data("b") != Data("c")
# test_ne(Data("b"), Data("c"))
assert Data("b", {"name": "silva"}) != Data("b", {"name": "santos"})
# test_ne(Data("b", {"name": "silva"}), Data("b", {"name": "santos"}))

INFO:root:Setting parent [b {'age': 22}] for [d {}]
INFO:root:Setting parent [b {'age': 22}] for [d {'age': 22}]


You can test if an element is a child of another

In [13]:
test_eq(olive in james, True)

### Cloning

You can duplicate any `Data` instance

In [14]:
james.clone()

<person {'name': 'james', 'root': 'true'}>
    [person {'name': 'olive', 'root': 'true'}]
    [person {'name': 'silva', 'root': 'true'}]
    [person {'name': 'john', 'root': 'true'}]
</person>

In [15]:
# | hide

root = Data("root")
root.append(Data("child"))

boot = root
assert root == boot

root.attrs["extra"] = "please"
assert root == boot

shoe = root.clone()
shoe.attrs["extra"] = "please2"
assert root != shoe

INFO:root:Setting parent [root {}] for [child {}]


### Logic

#### Basic iteration of all the elements

If you just need to iterate through all the elements, a simple loop will suffice

In [16]:
for person, level in james:
    print("   " * level, person.name + "::" + person.attrs["name"])

 person::james
    person::olive
       person::andrew
       person::jane
       person::
    person::silva
    person::john


---

In [17]:
# | hide
import nbdev

nbdev.nbdev_export()